# Implement the use Auth only using phone number

In [4]:
from supabase import create_client
from typing import Dict, Any

class SupabasePhoneAuth:
    def __init__(self, supabase_url: str, supabase_key: str):
        self.supabase = create_client(supabase_url, supabase_key)
    
    def format_phone(self, phone: str) -> str:
        """Format phone number to ensure it starts with +"""
        return f"+{phone}" if not phone.startswith('+') else phone
    
    def request_otp(self, formatted_phone: str) -> Dict[str, Any]:
        """Request OTP for phone number"""
        try:
            # formatted_phone = self.format_phone(phone)
            
            response = self.supabase.auth.sign_in_with_otp({
                "phone": formatted_phone
            })
            
            return {
                "success": True,
                "message": "OTP sent successfully",
                "phone": formatted_phone
            }
            
        except Exception as e:
            return {
                "success": False,
                "error": str(e),
                "phone": formatted_phone
            }
    
    def verify_otp(self, phone: str, otp: str) -> Dict[str, Any]:
        """Verify OTP"""
        try:
            formatted_phone = self.format_phone(phone)
            
            response = self.supabase.auth.verify_otp({
                "phone": formatted_phone,
                "token": otp,
                "type": "sms"
            })
            
            # After successful verification, get the profile
            profile = self.get_profile_by_phone(formatted_phone)
            
            return {
                "success": True,
                "user": response.user,
                "session": response.session,
                "profile": profile.get("data")
            }
            
        except Exception as e:
            return {
                "success": False,
                "error": str(e)
            }

    def update_profile(self, user_id: str, data: Dict[str, Any]) -> Dict[str, Any]:
        """Update user profile"""
        try:
            response = self.supabase.table('profiles')\
                .update(data)\
                .eq('id', user_id)\
                .execute()
                
            return {
                "success": True,
                "data": response.data[0] if response.data else None
            }
            
        except Exception as e:
            return {
                "success": False,
                "error": str(e)
            }
    
    def get_profile(self, user_id: str) -> Dict[str, Any]:
        """Get user profile by user ID"""
        try:
            response = self.supabase.table('profiles')\
                .select('*')\
                .eq('id', user_id)\
                .single()\
                .execute()
                
            return {
                "success": True,
                "data": response.data
            }
            
        except Exception as e:
            return {
                "success": False,
                "error": str(e)
            }
    
    def get_profile_by_phone(self, phone: str) -> Dict[str, Any]:
        """Get user profile by phone number"""
        try:
            # Remove the '+' for comparison since we store it without '+' in the database
            phone_without_plus = phone.replace('+', '')
            
            response = self.supabase.table('profiles')\
                .select('*')\
                .eq('phone_number', phone_without_plus)\
                .single()\
                .execute()
                
            return {
                "success": True,
                "data": response.data
            }
            
        except Exception as e:
            return {
                "success": False,
                "error": str(e)
            }

In [13]:
import os
from dotenv import load_dotenv

load_dotenv()

# Initialize the auth handler
auth = SupabasePhoneAuth(
    supabase_url=os.getenv("SUPABASE_URL"),
    supabase_key=os.getenv("SUPABASE_KEY")
)

supabase = create_client(
    os.getenv("SUPABASE_URL"),
    os.getenv("SUPABASE_KEY")
)



In [14]:
supabase

In [16]:
res = supabase.auth.sign_in_with_otp({
    'phone' : '+12045585998',
})

AuthApiError: Database error saving new user

In [11]:
response = supabase.auth.verify_otp({
  'phone': '12045585998',
  'token': '123456',
  'type': 'sms',
})

AuthApiError: Token has expired or is invalid

In [6]:
# Step 1: Request OTP
result = auth.request_otp("12045585998")
print(result)

if result["success"]:
    # In a real application, you'd wait for the user to receive the OTP
    otp_code = input("Enter the OTP you received: ")
    
    # Step 2: Verify OTP
    verified = auth.verify_otp("12045585998", otp_code)
    print(verified)
    
    if verified["success"]:
        # Profile is automatically created by the trigger
        # You can update additional fields if needed
        profile_update = auth.update_profile(
            verified["user"].id,
            {
                "first_name": "John",
                "last_name": "Doe"
            }
        )
        print(profile_update)

# You can also lookup profiles by phone number
profile = auth.get_profile_by_phone("+12045585998")
print(profile)

{'success': False, 'error': 'Database error saving new user', 'phone': '12045585998'}
{'success': False, 'error': "{'code': 'PGRST116', 'details': 'The result contains 0 rows', 'hint': None, 'message': 'JSON object requested, multiple (or no) rows returned'}"}


In [25]:
result = auth_handler.request_otp("12045585998")  # without + prefix
print(result)


{'success': True, 'message': 'OTP sent successfully', 'phone': '+12045585998'}


In [27]:
# Step 2: Verify OTP (after user receives it)
verified = auth_handler.verify_otp("12045585998", "109321")
if verified["success"]:
    # Step 3: Update profile if needed
    profile_update = await auth_handler.update_profile(
        verified["user"].id,
        {
            "first_name": "John",
            "last_name": "Doe"
        }
    )
    print(profile_update)

In [21]:
# Verify OTP
verified

{'success': False,
 'error': "object AuthResponse can't be used in 'await' expression"}

In [16]:
verify_result

<coroutine object SupabasePhoneAuth.verify_otp at 0x0000025605865D90>